# Initial Setup

In [ ]:
import os
import numpy as np
import datetime
import time
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
#from tensorflow.keras.layers import *
#from tensorflow.keras.models import *
#import tensorflow_datasets as tfds
#from imutils import paths

#tfds.disable_progress_bar()

## Imagenet Subset 

In [ ]:
dataTrainDir = '/Users/rmunoz/Data/imagenet-5classes/train'
dataValDir = '/Users/rmunoz/Data/imagenet-5classes/val'

configDir = 'config'
resultsDir = 'results'
modelsDir = 'models/imagenet_xception'
logsDir = 'logs/imagenet_xception'

In [ ]:
NUM_GPUS = 1
BS_PER_GPU = 64
NUM_EPOCHS = 50
AUTO = tf.data.experimental.AUTOTUNE

NUM_CLASSES = 5
HEIGHT = 150
WIDTH = 150
NUM_CHANNELS = 3
IMAGE_SIZE = (HEIGHT, WIDTH)

In [ ]:
if not os.path.exists(configDir):
    os.makedirs(configDir)

if not os.path.exists(resultsDir):
    os.makedirs(resultsDir)

if not os.path.exists(modelsDir):
    os.makedirs(modelsDir)
        
if not os.path.exists(logsDir):
    os.makedirs(logsDir)

In [ ]:
labelsFile = os.path.join(configDir, 'labels_imagenet_5classes.csv')

labelsName = next(os.walk(dataTrainDir))[1]
labelsName.sort()
print("Number of folders in TrainDir: {}".format(len(labelsName)))

labelsDF = pd.DataFrame()
labelsDF['name'] = labelsName
labelsDF['label'] = range(len(labelsName))

labelsDF.to_csv(labelsFile, index=False)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataTrainDir,
    labels='inferred',
    label_mode='int',
    class_names=labelsName,
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BS_PER_GPU,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataValDir,
    labels='inferred',
    label_mode='int',
    class_names=labelsName,
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BS_PER_GPU,
)

## Visualize data

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(labelsName[int(labels[i])])
        plt.axis("off")

## Using image data augmentation

When you don't have a large image dataset, it's a good practice to artificially introduce sample diversity by applying random yet realistic transformations to the training images, such as random horizontal flipping or small random rotations. This helps expose the model to different aspects of the training data while slowing down overfitting.

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
augmented_train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

## Configure the dataset for performance

Let's make sure to use buffered prefetching so we can yield data from disk without having I/O becoming blocking:

In [ ]:
augmented_train_ds = augmented_train_ds.prefetch(buffer_size=AUTO)
val_ds = val_ds.prefetch(buffer_size=AUTO)

## Create model

In [ ]:
modelDiagramFile = os.path.join(resultsDir, 'model_resnet50_imagenet_5classes.png')

resnet50 = tf.keras.applications.ResNet50(weights=None, include_top=False)
model = tf.keras.Sequential([resnet50, layers.GlobalAveragePooling2D(), layers.Dropout(0.25), layers.Dense(NUM_CLASSES, activation='softmax')])

keras.utils.plot_model(model, to_file=modelDiagramFile, show_shapes=True)

## Train model

In [ ]:
decay_steps = 1000
lr_decayed_fn = keras.experimental.CosineDecay(
    initial_learning_rate=0.001, decay_steps=decay_steps)

model.compile(optimizer=keras.optimizers.Adam(lr_decayed_fn),
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])

In [ ]:
logsCheckDir = os.path.join(logsDir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    
callbacks = [
    keras.callbacks.ModelCheckpoint(os.path.join(modelsDir, "model_checkpoint_{epoch}.h5")),
    keras.callbacks.TensorBoard(log_dir=logsCheckDir, histogram_freq=1),
    keras.callbacks.EarlyStopping(monitor="val_sparse_categorical_accuracy", patience=5, restore_best_weights=True, verbose=2)
]

startTime = time.time()

model.fit(augmented_train_ds,
          epochs=NUM_EPOCHS,
          callbacks=callbacks,
          validation_data=val_ds,
          validation_freq=1)

endTime = time.time()

elapsedTime = (endTime - startTime)/60.
print("\nTotal time for model training: {} minutes".format(elapsedTime))

In [ ]:
modelFile = os.path.join(modelsDir, 'model_resnet50_imagenet_5classes.h5')

model.save(modelFile)